# Ejercicio 4 - guía 3

En este ejercicio, utilizaremos el dataset de automóviles disponible en el siguiente enlace:
https://archive.ics.uci.edu/static/public/9/auto+mpg.zip. Este conjunto de datos contiene información sobre varios atributos de automóviles, incluyendo Millas por galón de combustible (MPG) y Caballos de Fuerza (HP).

## a)
Ajustar un modelo lineal que relacione MPG con HP utilizando todos los puntos del dataset.
Calcular el 𝑅 cuadrado para este modelo.

### Desarrollo

Primero leo los datos del archivo csv:

In [15]:
import pandas as pd
import os

ruta_datos_autos = os.path.join('..', '..','datos', 'raw' ,'auto-mpg.csv')

df_autos = pd.read_csv(ruta_datos_autos)

df_autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


De la información del dataframe se puede ver que horsepower no está en el tipo adecuado, por lo tanto procedo a convertirlo:

In [16]:
df_autos['horsepower'] = pd.to_numeric(df_autos['horsepower'], errors='coerce')

Verifico valores nulos en las variables de interés:

In [17]:
df_autos['horsepower'].isnull().sum()

np.int64(6)

In [18]:
df_autos['mpg'].isnull().sum()

np.int64(0)

Elimino todos los valores nulos del dataframe:

In [19]:
# elimino todas las filas con valores nulos
df_autos = df_autos.dropna()

Ahora ajusto el modelo lineal:

In [20]:
from utils.regresiones import generar_regresion_lineal_simple
import numpy as np

# Hago una estimación de los coeficientes
B1, B0, e = generar_regresion_lineal_simple(X = df_autos['mpg'], Y = df_autos['horsepower'])

print(f'Coeficientes del ajuste lineal= {B1}, B0 = {B0}, error medio = {np.mean(e)}')

Coeficientes del ajuste lineal= -3.838888032658009, B0 = 194.47564319018682, error medio = -7.250436079184696e-15


Me armo un subdataframe en donde voy a agrupar a la variable que usé como predictora, la variable objetivo y las predicciones junto con su error de cálculo:

In [21]:
df_experimento = pd.DataFrame({
    'horsepower': df_autos['horsepower'],
    'MPG': df_autos['mpg'],
    'Prediccion': B0 + B1*df_autos['mpg'],
    'Error': e
})

df_experimento.head()

,horsepower,MPG,Prediccion,Error
0,130.0,18.0,125.375659,4.624341
1,165.0,15.0,136.892323,28.107677
2,150.0,18.0,125.375659,24.624341
3,150.0,16.0,133.053435,16.946565
4,140.0,17.0,129.214547,10.785453


Ahora calculo el R cuadrado:

In [22]:
def calcular_r2(Y: pd.Series, Y_pred: pd.Series) -> float:
    '''
    Calcula el coeficiente de determinación de un modelo de regresión lineal
    '''
    
    TSS = sum((Y - Y.mean())**2)
    RSS = sum((Y - Y_pred)**2)
    
    return 1 - RSS/TSS

# Calculo el coeficiente de determinación de nuestra regresión calculada
r2 = calcular_r2(df_experimento['horsepower'], df_experimento['Prediccion'])
print(f'El coeficiente de determinación es {r2}')

El coeficiente de determinación es 0.6059482578894362


### (i)
Ajustar logaritmo de MPG vs HP y calcular 𝑅2.

Añado una nueva columna al dataframe que creé tomando el logaritmo de MPG, genero el modelo y calculo el R cuadrado:

In [23]:
df_experimento['log_MPG'] = np.log(df_experimento['MPG'])

B1_log_log, B0_log_log, e_log_log = generar_regresion_lineal_simple(X = df_experimento['log_MPG'], Y = df_experimento['horsepower'])

print(f'Coeficientes del ajuste lineal= {B1_log_log}, B0 = {B0_log_log}, R2 = {calcular_r2(df_experimento["horsepower"], B0_log_log + B1_log_log*df_experimento["log_MPG"])}')

Coeficientes del ajuste lineal= -93.97050440243893, B0 = 395.61941876339455, R2 = 0.6891575108068915


### (ii)
Ajustar logaritmo de MPG vs logaritmo HP y calcular 𝑅2.

Sigo el mismo procedimiento que antes pero añadiendo una nueva columna al dataframe tomando logaritmo de horsepower:

In [24]:
df_experimento['log_horsepower'] = np.log(df_experimento['horsepower'])

B1_log_log, B0_log_log, e_log_log = generar_regresion_lineal_simple(X = df_experimento['log_MPG'], Y = df_experimento['log_horsepower'])

print(f'Coeficientes del ajuste lineal= {B1_log_log}, B0 = {B0_log_log}, R2 = {calcular_r2(df_experimento["log_horsepower"], B0_log_log + B1_log_log*df_experimento["log_MPG"])}')

Coeficientes del ajuste lineal= -0.8584656841907069, B0 = 7.24772673162475, R2 = 0.7226967512359872


## b)
Veamos qué pasa si no usamos todo el dataset sino sólo un porcentaje.

### (i)
Ajustar un modelo lineal a MPG vs HP, pero esta vez utilizando solo el 80% de los puntos
del dataset seleccionados al azar. ¿Cuánto vale 𝑅2?, ¿Y sobre el 20% restante de los
puntos?

Separo los datos del dataframe entre 80% y 20% para ajustar los dos modelos y compararlos:

In [25]:
# Tomo un 80% de los datos aleatoriamente y dejo el otro 20% en otro dataframe
df_80 = df_experimento.sample(frac=0.8)
df_20 = df_experimento.drop(df_80.index)

# Ajusto dos modelos de MPG vs horsepower con los dos dataframes
B1_80, B0_80, e_80 = generar_regresion_lineal_simple(X = df_80['MPG'], Y = df_80['horsepower'])
B1_20, B0_20, e_20 = generar_regresion_lineal_simple(X = df_20['MPG'], Y = df_20['horsepower'])

print(f'Coeficientes del ajuste lineal= {B1_80}, B0 = {B0_80}, R2 = {calcular_r2(df_80["horsepower"], B0_80 + B1_80*df_80["MPG"])}')
print(f'Coeficientes del ajuste lineal= {B1_20}, B0 = {B0_20}, R2 = {calcular_r2(df_20["horsepower"], B0_20 + B1_20*df_20["MPG"])}')

Coeficientes del ajuste lineal= -3.6831182952011883, B0 = 189.53252714587018, R2 = 0.6099365420676865
Coeficientes del ajuste lineal= -4.398320960155555, B0 = 212.71670046599036, R2 = 0.6172997809502476


In [26]:
df_autos

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger


## c)
Utilizando el dataset completo, realizar ajustes lineales para relacionar MPG con cada una
de las variables. Ordenar las variables de acuerdo al 𝑅2 obtenido de la más importante a la
menos importante.

In [27]:
# Tomo MPG como variable dependiente y el resto como independientes
X = df_autos[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin']]
Y = df_autos['mpg']

# Armo una para guardar los datos de cada modelo
modelos = []

# Ajusto un modelo de regresión lineal simple para cada variable independiente
for var in X.columns:
    B1, B0, e = generar_regresion_lineal_simple(X = X[var], Y = Y)
    r2 = calcular_r2(Y, B0 + B1*X[var])
    modelos.append({
        'Var_Objetivo': 'mpg',
        'Var_Independiente': var,
        'B0': B0,
        'B1': B1,
        'R2': r2
    })

# Convierto la lista de diccionarios en un DataFrame
df_modelos = pd.DataFrame(modelos)
df_modelos.sort_values('R2', ascending=False)

,Var_Objetivo,Var_Independiente,B0,B1,R2
3,mpg,weight,46.216525,-0.007647,0.692630
1,mpg,displacement,35.120636,-0.060051,0.648229
2,mpg,horsepower,39.935861,-0.157845,0.605948
0,mpg,cylinders,42.915505,-3.558078,0.604689
5,mpg,model year,-70.011674,1.230035,0.337028
6,mpg,origin,14.811974,5.476547,0.319461
4,mpg,acceleration,4.833250,1.197624,0.179207


## Guardo el archivo para usarlo en posteriores ejercicios

In [28]:
ruta_guardado_autos = os.path.join('..', '..','datos', 'processed' ,'auto-mpg-processed.csv')

df_autos.to_csv(ruta_guardado_autos, index=False)